In [1]:
######################################################################
#
# Name: eQTL Pre-Processing
# Author: Gabe Foster
# Date: 5/15/2020
# Purpose: This script performs the initial preprocessing of the 
# NF54HT-GFP-luc x NHP4026 transcriptional cross section
#
######################################################################

import pandas as pd
import numpy as np
import re


In [113]:
######################################################################
#
# Read block- let's get our file, set our variables, and perform the
# initial filtering
#
######################################################################

rawcounts = pd.read_csv('GENE2.count',
                       sep = '\t')

# Setting fragment length- we can't see genes smaller than this and
# calculating TPM correctly requires it. Tx ran this as 250bp PE,
# and I don't have the original data so we're ballparking at 250.

fraglength = 250

# Remove genes shorter than frag length

rawcounts_filt = rawcounts[rawcounts['Length'] > fraglength].T


# We have a mess of strain name issues to fix here, as well

rawcounts_filt.index = rawcounts_filt.index.str.replace('\/','', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('ND5A5', 'AC075', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('ND6G8', 'AC125', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('N1', '', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('\\.', '', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('_4026', '_NHP4026', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('^4026', 'NHP4026', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('2H9', 'AC030', regex = True)
rawcounts_filt.index = rawcounts_filt.index.str.replace('6E5', 'AC033', regex = True)


In [5]:
######################################################################
#
# Initial sample QC- drop samples with < 3000 genes represented, zero
# any sample/gene reads < 5, and pick rep with most counts
#
######################################################################

# Keep only samples with counts in > 3000 genes, and sum counts for
# each sample along the way
good_samples = []
sample_sum = []
for i in range(6,len(rawcounts_filt.index)):
    genes_rep = np.sum(rawcounts_filt.iloc[i,:] > 0)
    if genes_rep > 3000:
        good_samples.append(rawcounts_filt.index[i])
        sample_sum.append(np.sum(rawcounts_filt.iloc[i,:]))
        
# Build summary frame, iterate over samples by strain/hpi and choose 
# the sample with the most reads to keep

count_summary = {'Sample Name':good_samples,
                 'Total Counts':sample_sum}
count_summary = pd.DataFrame(count_summary)

# This is a hot mess of regular expression that just converts the entire
# sample name to strain_##, where ## is the sampling timepoint. The formatting
# is very inconsistent throughout, so a mess of replacements need to be carefully
# made. This mess of substitutions makes them.

count_summary['Strain and Time'] = count_summary['Sample Name'].str.replace('^GF_PL[\d]+[a,b,c]{0,1}_', '',regex = True)
count_summary['Strain and Time'] = count_summary['Strain and Time'].str.replace('[A,B,C]_', '', regex = True)
count_summary['Strain and Time'] = count_summary['Strain and Time'].str.replace('_[d]+$', '', regex = True)
count_summary['Strain and Time'] = count_summary['Strain and Time'].str.replace('_S.*', '', regex = True)
count_summary['Strain and Time'] = count_summary['Strain and Time'].str.replace('_[0-9]{3,4}$', '', regex = True)
count_summary['Strain and Time'] = count_summary['Strain and Time'].str.replace('hpi', '', regex = True)
count_summary['Strain and Time'] = count_summary['Strain and Time'].str.replace('_T', '_', regex = True)

# Believe it or not, the GTEx consortium handled reps across batches and such by...
# taking the replicate with the most counts. That's easy, let's do that.

best_samples = []
for sample in count_summary['Strain and Time'].unique():
    subframe = count_summary[count_summary['Strain and Time'] == sample]
    best_samples.append(count_summary.iloc[subframe['Total Counts'].idxmax(),:])
    
# Now I'll build a frame with just the best samples in it
    
best_samples = pd.DataFrame(best_samples)
best_samplenames = ['Geneid', 'Chr', 'Start', 'End', 'Strand', 'Length']
best_samplenames.extend(list(best_samples['Sample Name']))
curated_counts = rawcounts_filt[rawcounts_filt.index.isin(best_samplenames)]


In [285]:
rawcounts = pd.read_csv('T4_counts.txt', 
                          sep = '\t', index_col=0)
rawcounts

,AC033,AC004,AC006,AC007,AC008,AC025,AC027,AC028,AC034,AC049,...,AC118,AC081,AC038,AC082,AC030,AC032,AC074,AC130,NF54gfp,AC075
Geneid,,,,,,,,,,,,,,,,,,,,,
malmito_rna_16:rRNA,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
malmito_rna_20:rRNA,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,3,1,1,0,0
malmito_rna_9:rRNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
malmito_rna_17:rRNA,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0
malmito_rna_LSUC:rRNA,0,0,0,0,0,0,0,0,2,0,...,3,0,0,0,0,4,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF3D7_1479700,0,0,0,1,4,0,0,0,3,0,...,0,0,1,0,2,0,1,0,1,0
PF3D7_1479800,0,0,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,1,0,1,0
PF3D7_1479900,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [286]:
def sample_and_geneqc(expdata):
    '''
    Here, we'll remove poor samples and poor genes, and QC low read #s
    Poor samples are defined as those with < 3000 genes with reads
    Poor genes are those that appear in < 20% of samples
    Low reads are any gene/sample reading < 5 counts; those are zeroed out
    '''
    
    # Curate Samples
    
    genecounts = pd.Series(data = np.count_nonzero(expdata, axis = 1),
                             index = expdata.index)
    samplecounts = pd.Series(np.count_nonzero(expdata,axis = 0),
                          index = expdata.columns)
    
    goodgenes = genecounts[genecounts/samplecounts.size > 0.2]
    
    goodsamples = samplecounts[samplecounts > 3000]
    
    allcur = expdata.loc[goodgenes.index, goodsamples.index]
    
    # Zero out counts < 5
    
    allcur = allcur.mask(allcur < 5, 0)
    
    return allcur

In [287]:
test = sample_and_geneqc(rawcounts)
test

,AC033,AC004,AC006,AC007,AC008,AC025,AC027,AC028,AC034,AC049,...,AC118,AC081,AC038,AC082,AC030,AC032,AC074,AC130,NF54gfp,AC075
Geneid,,,,,,,,,,,,,,,,,,,,,
malmito_rna_20:rRNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
malmito_rna_17:rRNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
malmito_rna_LSUC:rRNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
malmito_rna_LSUG:rRNA,0,7,5,0,0,0,0,0,6,0,...,15,0,0,6,5,12,0,0,10,0
malmito_SSUB:rRNA,0,5,0,0,0,0,0,0,0,0,...,5,0,0,0,0,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF3D7_1479200,0,5,0,5,15,12,33,49,0,28,...,6,0,8,34,5,9,22,24,14,21
PF3D7_1479400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PF3D7_1479700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [383]:
def tpm_norm(gctdata, probefile, fraglength = 250):
    '''
    TPM is (reads / effective gene length (in kb)) / (sample reads / 1e6)
    The elaborate nonsense below just calculates this, it's a bit of a
    mess but hey that's programming
    '''
    
    # Read in probe metadata, subtract frag length,
    # and convert to dict
    
    
    probeinfo = pd.read_csv(probefile, usecols = [1,6])
    probeinfo.index = probeinfo['Geneid']
    probeinfo.drop(columns = 'Geneid', inplace = True)
    probeinfo = probeinfo[probeinfo.index.isin(gctdata.index)]
    probeinfo['Length'] = probeinfo['Length'].apply(lambda x: x-fraglength)
        
    # Build output frame from input frame (cheating), calc length and
    # build lookup from samplename to total counts
    
    tpm = gctdata.copy()
    curated_tpm = gctdata.copy()
    
    # Iterate over rows- if non sample rows, just copy data, if sample
    # rows, calculate TPM
    
    # Note: it really doesn't like the way I did this and will throw
    # warnings- it does work, it's fine, don't worry about it, if you
    # are smarter than I am go ahead and rewrite it
    
    for (col, data) in gctdata.iteritems():
        
        numreads = data.sum()
        
        tempframe = pd.concat([probeinfo, data], axis = 1)

        tempframe['rpkb'] = tempframe.iloc[:,1].divide(tempframe['Length'].divide(1000))
        tempframe['tpm'] = (tempframe['rpkb'] / data.sum()) * 1e6
        tempframe['tpm'] = tempframe['tpm'].clip(lower = 0)
        curated_tpm.loc[:,col] = tempframe['tpm']
    return curated_tpm

In [384]:
test2 = tpm_norm(test, 'GENE2_probeinfo.csv', 250)
test2

,AC033,AC004,AC006,AC007,AC008,AC025,AC027,AC028,AC034,AC049,...,AC118,AC081,AC038,AC082,AC030,AC032,AC074,AC130,NF54gfp,AC075
Geneid,,,,,,,,,,,,,,,,,,,,,
malmito_rna_20:rRNA,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.000,-0.000000,-0.00000,-0.0,-0.000000,...,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000
malmito_rna_17:rRNA,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.000,-0.000000,-0.00000,-0.0,-0.000000,...,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000
malmito_rna_LSUC:rRNA,-0.0,-0.000000,-0.0,-0.000000,-0.000000,-0.000,-0.000000,-0.00000,-0.0,-0.000000,...,-0.000000,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000
malmito_rna_LSUG:rRNA,-0.0,0.000000,0.0,-0.000000,-0.000000,-0.000,-0.000000,-0.00000,0.0,-0.000000,...,0.000000,-0.0,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.00000
malmito_SSUB:rRNA,-0.0,0.000000,-0.0,-0.000000,-0.000000,-0.000,-0.000000,-0.00000,-0.0,-0.000000,...,0.000000,-0.0,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF3D7_1479200,0.0,7.767031,0.0,9.115203,24.895135,18.831,33.382695,86.70573,0.0,65.149443,...,8.726986,0.0,11.155266,28.595928,4.282387,8.378456,25.614292,19.650779,14.324915,21.31184
PF3D7_1479400,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.00000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
PF3D7_1479700,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.00000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [307]:
def testfunc(expdata, probedata, fraglength = 250):
    return expdata

In [93]:
test2 = testfunc(test, 'GENE2_probeinfo.csv')
test2

,AC033,AC004,AC006,AC007,AC008,AC025,AC027,AC028,AC034,AC049,...,AC118,AC081,AC038,AC082,AC030,AC032,AC074,AC130,NF54gfp,AC075
Geneid,,,,,,,,,,,,,,,,,,,,,
malmito_rna_20:rRNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
malmito_rna_17:rRNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
malmito_rna_LSUC:rRNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
malmito_rna_LSUG:rRNA,0,7,5,0,0,0,0,0,6,0,...,15,0,0,6,5,12,0,0,10,0
malmito_SSUB:rRNA,0,5,0,0,0,0,0,0,0,0,...,5,0,0,0,0,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF3D7_1479200,0,5,0,5,15,12,33,49,0,28,...,6,0,8,34,5,9,22,24,14,21
PF3D7_1479400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PF3D7_1479700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [197]:
######################################################################
#
# Time to calculate TPM
#
######################################################################

# TPM is (reads / effective gene length (in kb)) / (sample reads / 1e6)
# The elaborate nonsense below just calculates this, it's a bit of a
# mess but hey that's programming

# Build output frame from input frame (cheating), calc length and
# build lookup from samplename to total counts
curated_counts.columns = curated_counts.loc['Geneid',:]
curated_tpm = curated_counts
length = curated_counts.iloc[5,:] - fraglength
sample_counts = pd.Series(list(best_samples['Total Counts']),
                          index = list(best_samples['Sample Name'])).to_dict()

# Iterate over rows- if non sample rows, just copy data, if sample
# rows, calculate TPM

# Note: it really doesn't like the way I did this and will throw
# warnings- it does work, it's fine, don't worry about it, if you
# are smarter than I am go ahead and rewrite it

for row, data in curated_counts.iterrows():
    if row in ['Geneid','Chr', 'Start', 'End', 'Strand', 'Length']:
        curated_tpm.loc[row,:] = data
    else:
        rpkb = data / (length/1000)
        tpm = (rpkb / sample_counts[row]) * 1e6
        curated_tpm.loc[row,:] = tpm



NameError: name 'curated_counts' is not defined

In [7]:
######################################################################
#
# Subset block
# I have 3 time points and I want both the count data and the TPM
# for all 3, only on the samples that passed QC. Let's do it.
#
######################################################################

T4_samples = best_samples[best_samples['Strain and Time'].str.contains('_4$', regex = True)]
T30_samples = best_samples[best_samples['Strain and Time'].str.contains('_30$', regex = True)]
T44_samples = best_samples[best_samples['Strain and Time'].str.contains('_44$', regex = True)]



# Let's pull out the count data for each time point
# Now that we're normalized we can minimize this to a pure count matrix

rawcounts_filt.columns = rawcounts_filt.loc['Geneid',:]
filler = np.repeat('filler', len(rawcounts_filt.columns))

T4_counts = rawcounts_filt[rawcounts_filt.index.isin(T4_samples['Sample Name'])].T
T4_counts.name = 'T4_counts'
T4_counts.insert(loc = 0,column = 'Filler',value = filler)
T30_counts = rawcounts_filt[rawcounts_filt.index.isin(T30_samples['Sample Name'])].T
T30_counts.name = 'T30_counts'
T30_counts.insert(loc = 0,column = 'Filler',value = filler)
T44_counts = rawcounts_filt[rawcounts_filt.index.isin(T44_samples['Sample Name'])].T
T44_counts.name = 'T44_counts'
T44_counts.insert(loc = 0,column = 'Filler',value = filler)

# Alright, same thing but for TPM

T4_tpm = curated_tpm[curated_tpm.index.isin(T4_samples['Sample Name'])].T
T4_tpm.name = 'T4_tpm'
T4_tpm.insert(loc = 0,column = 'Filler',value = filler)
T30_tpm = curated_tpm[curated_tpm.index.isin(T30_samples['Sample Name'])].T
T30_tpm.name = 'T30_tpm'
T30_tpm.insert(loc = 0,column = 'Filler',value = filler)
T44_tpm = curated_tpm[curated_tpm.index.isin(T44_samples['Sample Name'])].T
T44_tpm.name = 'T44_tpm'
T44_tpm.insert(loc = 0,column = 'Filler',value = filler)

In [8]:
######################################################################
#
# Writing all this to a .gct file is a hassle, let us embark on this
# hassle here
#
######################################################################

# Here's a function that will write the above data frames to acceptable
# gct format

def to_gct(file, expdata):
    mrow = 0
    mcol = 0
    row = len(expdata.index)
    col = len(expdata.columns)
    
    outfile = open(file, 'w')
    outfile.write(f'1.0\n')
    outfile.write(f'{row}\t{col}\n')
    outfile.close()
    expdata.to_csv(file,
                   mode = 'a',
                   header = True,
                   sep = '\t',
                   index_label = 'Gene ID',
                   encoding='utf-8')
    

    
# Build a list of the files we need to write out
    
files_to_write = [T4_counts,
                  T30_counts,
                  T44_counts,
                  T4_tpm,
                  T30_tpm, 
                  T44_tpm]

# So, we need to think ahead a bit- we need to strip down sample
# names to just strain so that the name are consistent across both
# the gct files and the vcf file for future use. So, this loop will
# take each frame, use a bunch of replacement regexes to strip the 
# column names to just strain ID, and write to gct.

for edata in files_to_write:
    edata.columns = edata.columns.str.replace('^GF[\d]*_', '', regex = True)
    edata.columns = edata.columns.str.replace('PL[\da-z]*_', '', regex = True)
    edata.columns = edata.columns.str.replace('[A-C]{1}_', '', regex = True)
    edata.columns = edata.columns.str.partition('_').to_frame().iloc[:,0]
    
    to_gct(file = f'{edata.name}.gct',
          expdata = edata)
    
    

In [386]:
def build_covariates(metadata):
    '''
    Our samples were run across different plates; this
    is the only real place I can account for technical
    variation. Our sampling batches are confounded with
    stage, so I am forced to rely on CRC and PEER to
    pull anything in there out. To run PEER, I need
    one-hot encoded plate data, so I do that here.
    '''
    encoding = pd.get_dummies(metadata['PlateID'])
    metadata = metadata.merge(encoding,
                              left_index = True, 
                              right_index = True)
    metadata.drop(columns = 'PlateID',
                 inplace = True)
    
    return metadata

In [387]:
metadata = pd.read_csv(f'GENE2_NAME.txt', sep = '\t')
metadata.columns = ['SampleID', 'PlateID', 'Strain', 'Sampling Time', 'Sample Number', 'No Clue', 'Same']
covariates = build_covariates(metadata)
covariates.drop(columns = ['Strain', 'Sampling Time', 'Sample Number', 'No Clue', 'Same'], inplace = True)
covariates    

,SampleID,PL01,PL02,PL03,PL04,PL05,PL05a,PL05b
0,2H9_T44_S98_L00,0,0,0,0,0,0,0
1,2H9_T4_S96_L00,0,0,0,0,0,0,0
2,4026_T30_S103_L00,0,0,0,0,0,0,0
3,4026_T44_S104_L00,0,0,0,0,0,0,0
4,4026_T4_S102_L00,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
443,ND5A5_T44_S112_L00,0,0,0,0,0,0,0
444,ND5A5_T4_S110_L00,0,0,0,0,0,0,0
445,ND6G8_T30_S114_L00,0,0,0,0,0,0,0
446,ND6G8_T44_S115_L00,0,0,0,0,0,0,0


In [391]:
 timepoint_times = ['T4_', 'T30_', 'T44_']
i = 0
for j in range(1,3):
      #metasub = covariates[covariates['SampleID'].isin(timepoint.columns)].T
        metasub.columns = metasub.iloc[0,:]
        metasub = strip_names(metasub).T
        metasub.to_csv(f'{args.data_path}{timepoint_times[i]}batchcov.csv')

NameError: name 'metasub' is not defined